# Homework Assignment 2: QR Factorization

One algorithmic idea in numerical linear algebra is more important ([one of the 10 most important algorithms of the 20th century](https://archive.siam.org/pdf/news/637.pdf)) than all the others: QR factorization. This decomposition plays a crucial role in many problems:
* Computing orthogonal bases in a linear space.
* The preprocessing step for the singular value decomposition (principal component analysis).
* The computation of eigenvectors and eigenvalues (covered in next week's lectures).
* Methods for least squares problems (overdetermined systems of linear equations).

Before tackling the task of implementing this method, let's first introduce the Python library that provides all the necessary linear algebra functionality.

In [1]:
import numpy as np

## Problem 1: Getting started with arrays (2 points)

Implement the two functions `checkerboard` and `transpose` below, see instructions in the corresponding docstrings. 

**IMPORTANT**: DO NOT USE THE BUILT-IN METHODS arr.transpose()  OR arr.T OR THE FUNCTION np.transpose(arr). CREATE IT YOURSELF FROM SCRATCH!

In [2]:

def checkerboard(n):
    """
    Returns a two dimensional n by n checkerboard array C
    that alternates between 0's and 1's, starting with a 0
    in the top-left corner. 
    
    Example
    ----------
    checkerboard(4) =
        [[0 1 0 1]
         [1 0 1 0]
         [0 1 0 1]
         [1 0 1 0]]
    
    Parameters
    ----------
        n (int): Order of the n by n array
        
    Returns
    -------
        C (ndarray): Array filled with a checkerboard pattern
    
    """
    
    # YOUR CODE HERE
    import numpy as np
    C=np.ones((n,n),dtype=int)
    for i in range(0,n):
        for j in range(0,n):
            if (i+j)%2==0:
                C[i,j]=0
    
    return(C)

    
    
def transpose(arr):
    """
    Returns the transpose of a two dimensional array arr 
    
    Parameters
    ----------
        arr (ndarray): Any arbitrary two-dimensional array
        
    Returns
    -------
        arr_T (ndarray): Two-dimensional array arr transposed

    """
    
    # YOUR CODE HERE
    n_r,n_c=arr.shape

    arr_new=np.zeros((n_c,n_r),dtype=int)
    for i in range(0,n_r):
        for j in range(0,n_c):
            arr_new[j,i]=arr[i,j]
    
    return(arr_new)
    

In [3]:
# Test case
C = checkerboard(5)
print(C, "\n")

arr = np.array([[1, 2, 3], 
                [4, 5, 6], 
                [7, 8, 9]])
print(transpose(arr), "\n")


[[0 1 0 1 0]
 [1 0 1 0 1]
 [0 1 0 1 0]
 [1 0 1 0 1]
 [0 1 0 1 0]] 

[[1 4 7]
 [2 5 8]
 [3 6 9]] 



Expected output:

    [[0 1 0 1 0]
     [1 0 1 0 1]
     [0 1 0 1 0]
     [1 0 1 0 1]
     [0 1 0 1 0]]
     
    [[1 4 7]
     [2 5 8]
     [3 6 9]] 

In [4]:
# AUTOGRADING for checkerboard
C = [[0, 1, 0, 1, 0],
     [1, 0, 1, 0, 1],
     [0, 1, 0, 1, 0],
     [1, 0, 1, 0, 1],
     [0, 1, 0, 1, 0]] 
assert np.allclose(C, checkerboard(5))


In [5]:
# AUTOGRADING for transpose
arr = arr = np.array([[1, 2, 3], 
                      [4, 5, 6], 
                      [7, 8, 9]])
T = arr.transpose()
assert np.allclose(T, transpose(arr))

arr2D = np.arange(1,25).reshape(4,6)
T = arr2D.transpose()
assert np.allclose(T, transpose(arr2D))

import inspect
sig = inspect.signature(transpose)       # get the signature from the function 
source = inspect.getsource(transpose)    # get the source from the function (as a string! Wooow!)

assert not "np.transpose" in source
assert not "arr.transpose" in source
assert not "arr.T" in source

## Problem 2: (Classical) Gram-Schmidt Orthogonalization (3 points)

Implement the described Gram-Schmidt algorithm as a function `gram_schmidt_qr` below that takes a rectangular array $A$ and outputs $Q$ and $R$. 

**IMPORTANT**: Do not use any built-in QR functions like `np.linalg.qr` or `scipy.lingalg.qr`. These will give different results anyway as these use a different numerical scheme. 

**HINT**: The following functions for vectors might be useful: `np.linalg.norm` to compute the norm of a 1-D array and `np.inner` to compute the regular dot product between two 1-D arrays.

In [6]:

def gram_schmidt_qr(A):
    """
    Computes the QR factorization of a matrix A with linearly independent columns
    by means of the classical Gram-Schmidt process, where Q is an orthogonal matrix 
    and R is upper-triangular with positive entries on its diagonal. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by m
        R (ndarray): A two-dimensional array of size m by m
        
    """
    
    n, m = A.shape
    Q = np.zeros((n,m))
    R = np.zeros((m,m))
    
    # YOUR CODE HERE
    R[0,0]=np.linalg.norm(A[:,0])
    Q[:,0]=A[:,0]/R[0,0]
    for j in range(1,m):
        for i in range(j):
            R[i,j]=np.inner(A[:,j],Q[:,i])
        
        s=0
        for i in range(j):
            s+=R[i,j]*Q[:,i]
        perp_v_j=A[:,j]-s
        R[j,j]=np.linalg.norm(perp_v_j)
        Q[:,j]=perp_v_j/R[j,j]
    
    return Q, R


In [7]:
# Test case I
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this

A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
print(Q, "\n")
print(R)

[[ 0.66666667 -0.66666667]
 [ 0.33333333  0.66666667]
 [-0.66666667 -0.33333333]] 

[[3. 9.]
 [0. 6.]]


Expected output:

    [[ 0.66666667 -0.66666667]
     [ 0.33333333  0.66666667]
     [-0.66666667 -0.33333333]]
     
    [[3. 9.]
     [0. 6.]]

In [8]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = gram_schmidt_qr(B)
print(Q, "\n")
print(R)

[[-0.5  0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0.5]
 [-0.5  0.5 -0.5]] 

[[ 2. -5.  2.]
 [ 0.  3. -2.]
 [ 0.  0.  2.]]


Expected output:

    [[-0.5  0.5  0.5]
     [ 0.5  0.5 -0.5]
     [ 0.5  0.5  0.5]
     [-0.5  0.5 -0.5]]
     
    [[ 2. -5.  2.]
     [ 0.  3. -2.]
     [ 0.  0.  2.]]

In [9]:
# AUTOGRADER
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
Q_exact, R_exact = 1/3*np.array([[2, -2], [1, 2], [-2, -1]]), np.array([[3, 9], [0, 6]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)


## Problem 3: Householder QR Factorization (5 = 2 + 3 points)

Implement both the functions `householder_reflector` and `householder_qr` below. 

**Hint**: The function [`np.outer`](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) might be useful to compute $v v^T$ when we represent $v$ as a 1 dimensional array. The function [`np.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) is useful to compute the norm of a vector. The function [`np.inner`](https://numpy.org/doc/stable/reference/generated/numpy.inner.html) is useful to compute the dot product of two vectors. 

You can multiply two 2 dimensional arrays (i.e. matrices) $A$ and $B$ by `A @ B` or by `np.matmul(A,B)`. You can compute the inverse of a matrix by [`np.lingalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html).

In [10]:

def sign(x):
    """
    To be used in householder_qr below in the computation of
        v = sign(x) * norm(x) * e_1 + x
    """
    if(x >= 0):
        return(1)
    else:
        return(-1)

def householder_reflector(v, k, n):
    """
    Returns the orthogonal n by n Householder reflector Q_k as described by equation (1) above,
    corresponding to the hyperplane determined by the vector v.
    
    Parameters
    ----------
        v (ndarray): A 1 dimensional array of length n-k+1. 
        k (int): Denotes the required Householder reflector
                 BEWARE THAT PYTHON INDEXING STARTS AT ZERO,
                 THIS MIGHT BE CONFUSING WITH THE WAY Q_K IS
                 NUMBERED.
        n (int): Number of rows and columns of Q_k
    
    Returns
    -------
        Q_k (ndarray): A 2 dimensional n by n array. 
    """
    
    # YOUR CODE HERE
    
    v_t=np.transpose(v)
    
    F=np.eye(n-k+1)-2*(np.outer(v,v_t))/(np.matmul(v_t,v))
    Q_k=np.eye(n)
    Q_k[(k-1):n,(k-1):n]=F
    return(Q_k)
    

    
def householder_qr(A):
    """
    Computes a QR factorization of a matrix A by means of Householder
    reflectors, where Q is an orthogonal matrix and R is upper-triangular. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by n
        R (ndarray): A two-dimensional array of size n by m
    
    """
    
    # YOUR CODE HERE
    n,m= A.shape
    A_n=A
    Q_list=[]
    for k in range(m):
        x=A_n[k:,k]
        v=np.concatenate(([sign(x[0])*np.linalg.norm(x)],np.zeros(n-k-1)))+x
        Q_k=householder_reflector(v,k+1,n)
        A_n=np.matmul(Q_k,A_n)# this is the rolling product on the left,eventually =R
        Q_list.append(np.transpose(Q_k))#this is a list to calc the Q that we need 

    Q_final=Q_list[m-1] # now we need to reverse and multiply to get Q 
    for k in range(m-1):
        Q_final=np.matmul(Q_list[m-2-k],Q_final)

    return(Q_final,A_n)
    

In [11]:
# Test case for householder_reflector
v2 = np.array([10.8, -3.6])
Q_2 = householder_reflector(v2, 2, 3)
print(Q_2)

[[ 1.   0.   0. ]
 [ 0.  -0.8  0.6]
 [ 0.   0.6  0.8]]


In [12]:
# Test case for householder_qr
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this

A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = householder_qr(A)
print(Q, "\n")
print(R)

[[-0.66666667  0.66666667  0.33333333]
 [-0.33333333 -0.66666667  0.66666667]
 [ 0.66666667  0.33333333  0.66666667]] 

[[-3. -9.]
 [-0. -6.]
 [-0.  0.]]


Expected output:

    [[-0.66666667  0.66666667  0.33333333]
     [-0.33333333 -0.66666667  0.66666667]
     [ 0.66666667  0.33333333  0.66666667]]
     
    [[-3. -9.]
     [-0. -6.]
     [-0.  0.]]

In [13]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = householder_qr(B)
print(Q, "\n")
print(R)

[[-0.5 -0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]
 [-0.5 -0.5  0.5  0.5]] 

[[ 2. -5.  2.]
 [-0. -3.  2.]
 [-0. -0. -2.]
 [-0.  0.  0.]]


Expected output:

    [[-0.5 -0.5 -0.5 -0.5]
     [ 0.5 -0.5  0.5 -0.5]
     [ 0.5 -0.5 -0.5  0.5]
     [-0.5 -0.5  0.5  0.5]]
     
    [[ 2. -5.  2.]
     [ 0. -3.  2.]
     [ 0.  0. -2.]
     [ 0. -0.  0.]]

In [14]:
# AUTOGRADER for householder_reflector
v1 = np.array([5, 1, -2])
Q_1 = householder_reflector(v1, 1, 3)
assert np.allclose(Q_1, 1/15 * np.array([[-10, -5, 10], [-5, 14, 2], [10, 2, 11]]))

v2 = np.array([10.8, -3.6])
Q_2 = householder_reflector(v2, 2, 3)
assert np.allclose(Q_2, 1/5 * np.array([[5, 0, 0], [0, -4, 3], [0, 3, 4]]))

In [15]:
# AUTOGRADER for householder_qr
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = householder_qr(A)
Q_exact, R_exact = 1/3*np.array([[-2, 2, 1], [-1, -2, 2], [2, 1, 2]]), np.array([[-3, -9], [0, -6], [0, 0]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)
